# Imports

In [44]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta
import os
import matplotlib.pyplot as plt
import requests
from io import StringIO
from datetime import datetime

# Funciones

In [45]:
def get_nasdaq_tickers():
    url = 'http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt'
    response = requests.get(url)
    data = response.content.decode('utf-8')
    df = pd.read_csv(StringIO(data), sep='|')
    # Eliminar la última fila que contiene 'File Creation Time'
    df = df[:-1]
    # Asegurarse de que la columna 'Symbol' es de tipo cadena y eliminar valores nulos
    df['Symbol'] = df['Symbol'].astype(str)
    df = df[df['Symbol'].notnull()]
    tickers = df['Symbol'].tolist()
    # Filtrar los símbolos correctamente
    tickers = [
        ticker for ticker in tickers
        if isinstance(ticker, str) and 'test' not in ticker.lower() and ticker.isalpha()
    ]
    return tickers


In [46]:
def download_data(ticker, period='5y', interval='1d'):
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=False, progress=False)
    if df.empty:
        print(f"No hay datos para el ticker {ticker}")
        return None
    df.dropna(inplace=True)
    return df

def calculate_indicators(df, indicators, time_frame):
    df = df.copy()
    if time_frame == 'monthly':
        df = df.resample('M').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Adj Close': 'last',
            'Volume': 'sum'
        })
    elif time_frame == 'weekly':
        df = df.resample('W').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Adj Close': 'last',
            'Volume': 'sum'
        })
    df.dropna(inplace=True)
    # Calcular los indicadores seleccionados
    if 'coppock' in indicators:
        df['coppock'] = calculate_coppock_curve(df['Close'], wma_period=10, roc1_period=12, roc2_period=6)
    if 'macd' in indicators:
        macd = ta.trend.MACD(close=df['Close'], window_slow=23, window_fast=12, window_sign=9)
        df['macd'] = macd.macd()
        df['macd_signal'] = macd.macd_signal()
    if 'sma5' in indicators:
        df['sma5'] = ta.trend.SMAIndicator(close=df['Close'], window=5).sma_indicator()
    if 'sma20' in indicators:
        df['sma20'] = ta.trend.SMAIndicator(close=df['Close'], window=20).sma_indicator()
    if 'sma30' in indicators:
        df['sma30'] = ta.trend.SMAIndicator(close=df['Close'], window=30).sma_indicator()
    if 'sma50' in indicators:
        df['sma50'] = ta.trend.SMAIndicator(close=df['Close'], window=50).sma_indicator()
    if 'sma200' in indicators:
        df['sma200'] = ta.trend.SMAIndicator(close=df['Close'], window=200).sma_indicator()
    return df

def calculate_coppock_curve(series, wma_period=10, roc1_period=12, roc2_period=6):
    roc1 = series.pct_change(periods=roc1_period) * 100
    roc2 = series.pct_change(periods=roc2_period) * 100
    coppock = (roc1 + roc2).rolling(window=wma_period).mean()
    return coppock

def calculate_rsc_mansfield(df, benchmark_df):
    """
    Calcula la fuerza RSC Mansfield entre el activo y el índice de referencia.
    """
    ratio = df['Close'] / benchmark_df['Close']
    sma_ratio = ratio.rolling(window=52).mean()
    mansfield = (ratio / sma_ratio - 1) * 100
    return mansfield

In [48]:
def analyze_tickers(tickers):
    # Descargar datos del índice de referencia (por ejemplo, el Nasdaq Composite)
    benchmark_ticker = '^IXIC'  # Índice Nasdaq Composite
    benchmark_df = download_data(benchmark_ticker, period='5y', interval='1d')
    if benchmark_df is None:
        print("No se pudo descargar los datos del índice de referencia.")
        return

    results = []

    total_tickers = len(tickers)
    processed_tickers = 0

    for ticker in tickers:
        processed_tickers += 1
        print(f"Procesando ticker ({processed_tickers}/{total_tickers}): {ticker}")
        try:
            # Descargar datos diarios
            df_daily = download_data(ticker, period='1y', interval='1d')
            if df_daily is None or len(df_daily) < 200:
                continue  # Necesitamos al menos 200 días de datos para las SMA

            # Descargar datos semanales
            df_weekly = download_data(ticker, period='5y', interval='1d')
            if df_weekly is None or len(df_weekly) < 100:
                continue  # Necesitamos suficientes datos para calcular indicadores semanales
            df_weekly = calculate_indicators(df_weekly, indicators=['macd', 'sma5', 'sma20', 'sma30'], time_frame='weekly')
            if df_weekly.empty:
                continue

            # Descargar datos mensuales
            df_monthly = download_data(ticker, period='10y', interval='1d')
            if df_monthly is None or len(df_monthly) < 120:
                continue  # Necesitamos suficientes datos para calcular la Curva de Coppock
            df_monthly = calculate_indicators(df_monthly, indicators=['coppock'], time_frame='monthly')
            if df_monthly.empty:
                continue

            # Calcular SMA50 y SMA200 en datos diarios
            df_daily = calculate_indicators(df_daily, indicators=['sma50', 'sma200'], time_frame='daily')

            # Verificar que las SMA han sido calculadas
            if df_daily['sma50'].isnull().all() or df_daily['sma200'].isnull().all():
                continue

            # Señales de Compra y Venta

            ## Señales de Coppock
            # Señal de Compra
            coppock_current = df_monthly['coppock'].iloc[-1]
            coppock_prev = df_monthly['coppock'].iloc[-2]
            coppock_slope_positive = (coppock_current - coppock_prev) > 0
            coppock_below_zero = coppock_current < 0
            signal_coppock_buy = coppock_slope_positive and coppock_below_zero

            # Señal de Venta
            coppock_slope_negative = (coppock_current - coppock_prev) < 0
            signal_coppock_sell = coppock_slope_negative and coppock_current < 0

            ## Señales de MACD
            # Señal de Compra
            macd_signal_current = df_weekly['macd_signal'].iloc[-1]
            macd_signal_prev = df_weekly['macd_signal'].iloc[-2]
            macd_crosses_zero_up = (macd_signal_prev < 0) and (macd_signal_current > 0)
            macd_signal_positive = macd_signal_current > 0
            signal_macd_buy = macd_crosses_zero_up or macd_signal_positive

            # Señal de Venta
            macd_crosses_zero_down = (macd_signal_prev > 0) and (macd_signal_current < 0)
            macd_signal_negative = macd_signal_current < 0
            signal_macd_sell = macd_crosses_zero_down or macd_signal_negative

            ## Señales de Cruce de Medias
            # Señal de Compra
            sma50_current = df_daily['sma50'].iloc[-1]
            sma200_current = df_daily['sma200'].iloc[-1]
            golden_cross = sma50_current > sma200_current
            min_price = df_daily['Low'].iloc[-1]
            sma200_distance = abs(sma200_current - min_price) / min_price
            sma200_close_to_min = sma200_distance < 0.01  # Menor al 1%
            signal_sma_buy = golden_cross and sma200_close_to_min

            # Señal de Venta
            death_cross = sma50_current < sma200_current
            signal_sma_sell = death_cross

            ## Señales de Alfayate_Weinstein
            # Señal de Compra
            # Condición 1: Distancia del precio de cierre al máximo de 52 semanas <= 2%
            max_52weeks = df_daily['High'].rolling(window=252).max().iloc[-1]
            close_price = df_daily['Close'].iloc[-1]
            distance_to_max = abs(max_52weeks - close_price) / max_52weeks
            condition1_buy = distance_to_max <= 0.02

            # Condición 2: RSC Mansfield de 52 semanas > 0.10
            # Calcular RSC Mansfield para el activo
            rsc_mansfield = calculate_rsc_mansfield(df_weekly, benchmark_df)
            rsc_current = rsc_mansfield.iloc[-1]
            condition2_buy = rsc_current > 0.10

            # Condición 3: SMA5 del CPM52 >= 10
            df_weekly['CPM'] = df_weekly['Volume'] * df_weekly['Close']
            cpm52 = df_weekly['CPM'].rolling(window=52).mean()
            sma5_cpm52 = cpm52.rolling(window=5).mean()
            cpm52_current = sma5_cpm52.iloc[-1]
            condition3_buy = cpm52_current >= 10  # Ajusta el umbral según sea necesario

            # Condición 4: Riesgo stop <= 9%
            sma30_current = df_weekly['sma30'].iloc[-1]
            stop_risk = abs(close_price - sma30_current) / close_price
            condition4_buy = stop_risk <= 0.09

            # Señal de Compra Alfayate_Weinstein
            signal_alfayate_buy = condition1_buy and condition2_buy and condition3_buy and condition4_buy

            # Señal de Venta
            # Condición 1: RSC Mansfield de 52 semanas < -0.90
            condition1_sell = rsc_current < -0.90

            # Condición 2: SMA20 del CPM52 <= -15
            sma20_cpm52 = cpm52.rolling(window=20).mean()
            cpm52_current_sma20 = sma20_cpm52.iloc[-1]
            condition2_sell = cpm52_current_sma20 <= -15  # Ajusta el umbral según sea necesario

            # Condición 3: Riesgo stop >= 30%
            condition3_sell = stop_risk >= 0.30

            # Señal de Venta Alfayate_Weinstein
            signal_alfayate_sell = condition1_sell and condition2_sell and condition3_sell

            # Añadir los resultados para este ticker
            results.append({
                'Ticker': ticker,
                # Señales de Compra
                'Signal_Coppock_Buy': signal_coppock_buy,
                'Signal_MACD_Buy': signal_macd_buy,
                'Signal_SMA_Buy': signal_sma_buy,
                'Signal_Alfayate_Buy': signal_alfayate_buy,
                # Señales de Venta
                'Signal_Coppock_Sell': signal_coppock_sell,
                'Signal_MACD_Sell': signal_macd_sell,
                'Signal_SMA_Sell': signal_sma_sell,
                'Signal_Alfayate_Sell': signal_alfayate_sell,
                # Indicadores y valores calculados
                'Coppock': coppock_current,
                'MACD_Signal': macd_signal_current,
                'SMA50': sma50_current,
                'SMA200': sma200_current,
                'SMA200_Dist_Min': sma200_distance,
                'Distance_to_Max': distance_to_max,
                'RSC_Mansfield': rsc_current,
                'SMA5_CPM52': cpm52_current,
                'SMA20_CPM52': cpm52_current_sma20,
                'Stop_Risk': stop_risk
            })

        except Exception as e:
            print(f"Error al procesar {ticker}: {e}")
            continue

    # Convertir la lista de resultados en DataFrame
    results_df = pd.DataFrame(results)
    if not results_df.empty:
        results_df = results_df.set_index('Ticker')
        print("\nResultados del análisis:")
        display(results_df)
    else:
        print("\nNo se pudieron obtener resultados del análisis.")

    # Guardar el DataFrame en un archivo CSV con el formato especificado
    today = datetime.now().strftime('%Y%m%d')
    output_filename = f'analysis_results/resultados_{today}.csv'
    results_df.to_csv(output_filename)
    print(f"\nResultados guardados en: {output_filename}")

    return results_df

In [49]:
# Obtener la lista de tickers del NASDAQ
tickers = get_nasdaq_tickers()

# Debido a que el número de tickers es grande, puedes limitarlo para pruebas
#tickers = tickers[:100]  # Analizar solo los primeros 100 tickers

# Ejecutar el análisis
results_df = analyze_tickers(tickers)


Procesando ticker (1/4835): AACG


C:\Users\javip\AppData\Local\Temp\ipykernel_33412\106972179.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').agg({


Procesando ticker (2/4835): AADI


C:\Users\javip\AppData\Local\Temp\ipykernel_33412\106972179.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').agg({


Procesando ticker (3/4835): AADR


...

C:\Users\javip\AppData\Local\Temp\ipykernel_33412\106972179.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').agg({


Procesando ticker (4834/4835): ZYXI
Procesando ticker (4835/4835): ZZZ

Resultados del análisis:


C:\Users\javip\AppData\Local\Temp\ipykernel_33412\106972179.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample('M').agg({


,Signal_Coppock_Buy,Signal_MACD_Buy,Signal_SMA_Buy,Signal_Alfayate_Buy,Signal_Coppock_Sell,Signal_MACD_Sell,Signal_SMA_Sell,Signal_Alfayate_Sell,Coppock,MACD_Signal,SMA50,SMA200,SMA200_Dist_Min,Distance_to_Max,RSC_Mansfield,SMA5_CPM52,SMA20_CPM52,Stop_Risk
Ticker,,,,,,,,,,,,,,,,,,
AACG,True,False,False,False,False,True,True,False,-51.772105,-0.070628,0.665198,0.925149,0.015798,NaN,NaN,1.056339e+05,9.221979e+04,0.209826
AADI,True,False,False,False,False,True,True,False,-108.275102,-0.112272,1.747440,1.808285,0.095858,NaN,NaN,2.986558e+06,3.197897e+06,0.141093
AADR,False,True,False,False,False,False,False,False,38.964032,0.757198,62.073800,60.811650,0.059662,NaN,NaN,2.748577e+05,3.280018e+05,0.034828
AAL,True,False,False,False,False,True,True,False,-23.087360,-0.512465,10.694726,12.661031,0.082139,NaN,NaN,2.106545e+09,2.105658e+09,0.011542
AAME,False,False,False,False,True,True,True,False,-10.570678,-0.102781,1.609000,2.011950,0.169738,NaN,NaN,5.833491e+04,6.027768e+04,0.015891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTEK,False,False,False,False,True,True,True,False,-52.021712,-0.079778,0.828600,1.076600,0.888772,NaN,NaN,1.438161e+05,1.488008e+05,0.721053
ZUMZ,False,True,False,False,False,False,False,False,23.386217,1.352843,24.290600,19.607600,0.100197,NaN,NaN,2.468478e+07,2.375677e+07,0.053197
ZVRA,False,True,False,False,False,False,False,False,37.366240,0.473400,7.449300,5.983225,0.252097,NaN,NaN,1.355886e+07,8.922626e+06,0.259625



Resultados guardados en: analysis_results/resultados_20241014.csv


In [35]:
# Filtrar las filas donde al menos 2 de los 3 indicadores son True
filtered_results = results_df[(results_df[['Signal_Coppock', 'Signal_MACD', 'Signal_SMA']].sum(axis=1) >= 3)]

# Mostrar el resultado filtrado
filtered_results.shape

(13, 8)

In [37]:
filtered_results.head(13)

,Signal_Coppock,Signal_MACD,Signal_SMA,Coppock,MACD_Signal,SMA50,SMA200,SMA200_Dist_Min
Ticker,,,,,,,,
AHCO,True,True,True,-0.543338,0.241166,10.567800,9.768025,0.009328
CFFN,True,True,True,-5.970017,0.058964,5.780200,5.692900,0.004039
CLBK,True,True,True,-5.576747,0.134862,16.966400,16.635725,0.008226
CODX,True,True,True,-12.501981,0.023093,1.295000,1.237715,0.009828
DHIL,True,True,True,-13.656929,0.418694,155.607599,153.516300,0.001585
GOVI,True,True,True,-0.740014,0.265712,29.029800,28.109100,0.004283
HFBL,True,True,True,-26.612149,0.031736,12.764400,12.444350,0.002768
KRNY,True,True,True,-24.916843,0.031393,6.581400,6.480900,0.004470
LCNB,True,True,True,-5.648014,0.087439,14.864800,14.652150,0.009797
